In [1]:
%load_ext autoreload
%autoreload 2

$$\begin{aligned}
&\min_X f(X) = \textrm{tr}(A^\top XB X^\top)  \\
& = \textrm{tr}(X^\top A^\top XB) & x = \textrm{vec}(X)\\
& = \left <\textrm{vec}(X),  \textrm{vec}(A^\top X B )  \right > \\
& = \left <\textrm{vec}(X), B^\top \otimes A^\top \cdot \textrm{vec}(X)  \right > \\ 
& = x^\top (B^\top \otimes A^\top) x\\ 
& = \left<AX, XB\right> \\
\mathbf{s.t.} & \\ 
&X \in \Pi_{n}
\end{aligned}$$

In [2]:
from qap_lp.models import *

In [103]:
e.dot(e.T)

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [143]:
instance_name = 'chr12a'

A0, B0 = parse(f'qapdata/{instance_name}.dat')
A = A0 / np.linalg.norm(A0)
B = B0 / np.linalg.norm(B0)
n, m = A.shape
e = np.ones(shape=(n, 1))
E = np.ones(shape=(n, n))
ab = np.kron(B.T, A.T)

param = QAPParam(A, B, n, m, e, E, ab)

## projected gradient

working on problem:
### exact penalty

$$\begin{aligned}
F_{\mu} & =  f  + \mu\cdot | \textrm{tr}(XX^T) -  n| \\
 &= f  + \mu\cdot n - \mu\cdot \textrm{tr}(XX^T)
\end{aligned}$$

very likely to become a concave function, cannot be directly solved by conic solver.

#### derivatives

$$\begin{aligned}
\nabla F_\mu  = A^TXB + AXB^T - 2\mu X
\end{aligned}$$


#### projected derivative

$$\begin{aligned}
&\min_D ||\nabla F_\mu - D ||_F^2  \\
\mathsf{s.t.} & \\
&D e = D^\top e = 0 \\
\end{aligned}$$


In [83]:
nabla = QAPDerivativeL2Penalty(param)

In [455]:
def gt_min(x):
    return x[x >= -1e-4].min()

In [305]:
def pd_on_dc(param, dF, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_gradient_on_D_cone')
    D = model.variable("d", [*A.shape], dom.unbounded())
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, dF))
    model.constraint(expr.sum(D, 0), dom.equalsTo(0))
    model.constraint(expr.sum(D, 1), dom.equalsTo(0))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [335]:
def proj_on_dn(param, x, **kwargs):
    A, B, n, m, e, E, ab = param.A, param.B, param.n, param.m, param.e, param.E, param.ab
    
    model = mf.Model('projected_on_Dn')
    D = model.variable("d", [*A.shape], dom.inRange(0, 1))
    v = model.variable(1, dom.greaterThan(0.0))
    
    # m = vec(D - dF)
    m = expr.flatten(expr.sub(D, x))
    model.constraint(expr.sum(D, 0), dom.equalsTo(1))
    model.constraint(expr.sum(D, 1), dom.equalsTo(1))
    model.constraint(expr.vstack(v, m), dom.inQCone())
    model.objective(mf.ObjectiveSense.Minimize, v)

    # set params
    userCallback = set_mosek_model_params(model, **kwargs)
    model.setLogHandler(None)
    model.solve()

    model.flushSolutions()
    D_sol = D.level().reshape(A.shape)
    return D_sol

In [593]:
# initialize: x0 = [1]/n
x = x0 = np.ones((n, n)) / n
mu = 100
alpha = alpha0 = 0.1
nabla.obj(x, mu), nabla.original_obj(x)

(1100.2610504307938, 0.2610504307937287)

In [597]:
# point projection
for i in range(100):
    print(f'====={i }====')
    alpha = 1/(i+1)
    d0 = nabla.partial_f(x, mu)
    x0 = x - d0 * alpha0
    xp = proj_on_dn(param, x0)
#     obj = nabla.obj(x, mu)
#     # direction
#     dx = xp - x
#     st = gt_min(x/dx)
#     print(f"stepsize: ", alpha, st)
#     x -=  st * alpha * dx
    x = xp
    print(f"obj: ", nabla.obj(x, mu), nabla.obj(x, mu) - obj)
    if abs(n - x.dot(x.T).trace()) < 0.5:
        break
    print(f"stopping: ", n - x.dot(x.T).trace())
    print(x.round(3).min())

=====0====
obj:  0.25917477904181396 -10.89360776235451


In [598]:
xp.round(4)

array([[-0., -0., -0., -0., -0., -0.,  1., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  1., -0.],
       [ 1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.,  1.],
       [-0., -0., -0., -0., -0., -0., -0.,  1., -0., -0., -0., -0.],
       [-0., -0., -0., -0.,  1., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0.,  1., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0.,  1., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0.],
       [-0.,  1., -0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
       [-0., -0., -0., -0., -0., -0., -0., -0.,  1., -0., -0., -0.]])

In [458]:
# gradient projection, not good for now
for i in range(1):
    print(f'====={i }====')
    alpha *= 0.9
    d0 = nabla.partial_f(x, mu)
    dp = pd_on_dc(param, d0)
    ndf = np.linalg.norm(d0)
    print(f"norm: {ndf}")
    obj = nabla.obj(x, mu)
    # compute restricted stepsize
    st1 = gt_min(x/dp)
    print(f"stepsize: ", st1)
    x -=  st1 * dp
    print(f"obj: ", nabla.obj(x, mu), nabla.obj(x, mu) - obj)
    if abs(n - x.dot(x.T).trace()) < 1:
        break
    print(f"stopping: ", n - x.dot(x.T).trace())
    print(x.round(3).min())

=====0====
norm: 1.6733845869610735
stepsize:  0.0
obj:  11.152696760229476 0.0
stopping:  10.905510700018416
0.0


# eval

In [421]:
x_int, _ = extract_sol_rounding(x, param.A, param.B)

sampling results: min:0.10482158445449206, max:0.4129622032067326, avg:0.2619951630942756


In [419]:
def check_obj_val(x_sol):
    _obj = A0.T.dot(x_sol).dot(B0).dot(x_sol.T).trace()
    print(f'original obj {_obj}')
    return _obj

In [599]:
check_obj_val(x)

original obj 41064.000030433184


41064.000030433184